In [1]:
import numpy as np
import pandas as pd
import requests
import json

In [2]:
# load api keys from api_keys.txt file
mbta_api = 'https://api-v3.mbta.com/'

api_keys = dict()
with open('api_keys.txt', 'r') as f:
    for line in f.readlines():
        name, key = line.split('=')
        api_keys[name] = key.rstrip()

In [3]:
# setup google maps
import gmaps
import gmaps.datasets
gmaps.configure(api_key=api_keys['gmaps_api_key'])

In [4]:
r = requests.get(mbta_api+'routes/')
r.status_code

200

In [5]:
r.headers['content-type']
route_data = r.json()['data']

In [6]:
route_1 = [d for d in route_data if d['id'] == '1']
route_1_link = route_1[0]['links']['self']
route_1_data = requests.get('https://api-v3.mbta.com/routes/1')
route_1_data.json()

{'data': {'attributes': {'color': 'FFC72C',
   'description': 'Key Bus Route (Frequent Service)',
   'direction_names': ['Outbound', 'Inbound'],
   'long_name': '',
   'short_name': '1',
   'sort_order': 100,
   'text_color': '000000',
   'type': 3},
  'id': '1',
  'links': {'self': '/routes/1'},
  'type': 'route'},
 'jsonapi': {'version': '1.0'}}

In [7]:
vehicle_request = requests.get(mbta_api+'vehicles/').json()['data']

In [8]:
red_line_trains = [v for v in vehicle_request if v['relationships']['route']['data']['id'] == 'Red']
red_line_trains[0]

{'attributes': {'bearing': 290.0,
  'current_status': 'IN_TRANSIT_TO',
  'current_stop_sequence': 140,
  'direction_id': 1,
  'label': '1612',
  'last_updated': '2018-04-29T14:04:51-04:00',
  'latitude': 42.353111267089844,
  'longitude': -71.05616760253906,
  'speed': None},
 'id': 'R-54554657',
 'links': {'self': '/vehicles/R-54554657'},
 'relationships': {'route': {'data': {'id': 'Red', 'type': 'route'}},
  'stop': {'data': {'id': '70078', 'type': 'stop'}},
  'trip': {'data': {'id': 'ADDED-1524988080', 'type': 'trip'}}},
 'type': 'vehicle'}

In [9]:
red_line_train_coords = [ (train['attributes']['latitude'], train['attributes']['longitude']) for train in red_line_trains ]
rltc_array = np.asarray(red_line_train_coords)
rltc_array

array([[ 42.35311127, -71.0561676 ],
       [ 42.3670311 , -71.10626984],
       [ 42.29996872, -71.06181335],
       [ 42.30001831, -71.06182861],
       [ 42.3506012 , -71.05568695],
       [ 42.32268906, -71.05278778],
       [ 42.29592896, -71.04989624],
       [ 42.35649872, -71.0626297 ],
       [ 42.25202942, -71.00550842],
       [ 42.38825989, -71.11894989],
       [ 42.3961792 , -71.14054108],
       [ 42.24018097, -71.00518036],
       [ 42.36288834, -71.09249115]])

plot the train locations on a map

In [10]:
# create a gmaps symbol layer of red dots
trans_red = "rgba(200, 0, 0, 0.4)"
red_line_dots = gmaps.symbol_layer(
    rltc_array, fill_color=trans_red,
    stroke_color=trans_red, scale=3
)

In [23]:
figure_layout = {
    'width':'500px',
    'height':'500px'
}

In [24]:
fig=gmaps.figure(layout=figure_layout)
fig.add_layer(red_line_dots)
fig.add_layer(gmaps.transit_layer())
fig

Figure(layout=FigureLayout(height='500px', width='500px'))